<a href="https://colab.research.google.com/github/hokoro/Crawling/blob/main/naverapicrwainghomework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Client ID = 5MCGvUYn0TA9mdO1IRmK
## Client Pw = yX3wQfPqWP

### post 변수 링크 형식 예시 
- 'title': 'U-19 대표팀 김은중호, 신태용호와 친선전서 5-1 완승', 

- 'originallink': 'http://www.newsis.com/view/?id=NISX20220329_0001812653&cID=10503&pID=10500', 
- 'link': 'https://sports.news.naver.com/news.nhn?oid=003&aid=0011091271', 
- 'description': '아시안컵 예선을 통과하면 내년 열리는 아시안컵 본선에 나가고, 본선에서 4위 안에 들면 국제축구연맹(FIFA) U-20 <b>월드컵</b> 출전권을 딴다. 2023년 FIFA U-20 <b>월드컵</b>을 개최하는 인도네시아는 성공적인 대회를 치르기 위해... ', 'pubDate': 'Tue, 29 Mar 2022 20:55:00 +0900'}

In [ ]:
import os # 파일 관리 관련 모듈
import sys # 변수 함수 제어 관련 모듈
import urllib.request#url 열기 관련 모듈
import datetime # 일정 관련 모듈
import time #시간 관련 모듈
import json#json 파일 관련 모듈

In [ ]:
client_id = "5MCGvUYn0TA9mdO1IRmK" #발급받은 api _id
client_secret = "yX3wQfPqWP" #발급 받은 api_secret

In [ ]:
def getRequest(url):
  req = urllib.request.Request(url) #인자로 받은 url 을 통해 요청을 전송
  req.add_header("X-Naver-Client-Id", client_id) #url 요청에 다른 헤더를 추가 ID 추가
  req.add_header("X-Naver-Client-Secret", client_secret)#url 요청에 다른 헤더를 추가 secret 추가

  #try catch - url 성공적으로 열리는지 여부를 모르기때문에 예외처리로 에러를 해결
  try: #요청 받은 url 을 성공적으로 열경우 
    response = urllib.request.urlopen(req)  #url 을 성공적으로 열고 
    if response.getcode() == 200: #response의 HTTP status code 를 리턴한다.
      print("[%s] url Request Success" % datetime.datetime.now()) #url 요청을 성곡하면 datetime 을 통한 요청 성공 시간 반환
      return response.read().decode('utf-8') #한글을 읽어야 함으로 decode 를 utf-8 로 설정하여 읽는것을 반환 
  except Exception as e: #요청에 실패한경우
    print(e) #실패 에러 메시지 를 출력
    print("[%s] Error for URL: %s" %(datetime.datetime.now(),url)) #실패한 시간과 , url 을 출력
    return None #반환할게 없으므로 None

In [ ]:
def getNaverSearch(node,srcText,start,display):
  base = "https://openapi.naver.com/v1/search" #검색 api 를 사용한 기본 베이스 주소
  node = "/%s.json" % node #node = json 파일 명 
  parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(srcText),start,display) #우리가 검색할 키워드 가 담긴 이름
  url = base + node + parameters # url 을 만들기 위해 base + node + parameter 합치기 
  responseDecode = getRequest(url) #getRequestUrl 을 호출하여 해당 url 요청을 보냄 

  if (responseDecode == None): #요청에 실패한 경우
    return None
  else: #요청에 성공한 경우 
    return json.loads(responseDecode) #성공시 해당 검색 json 파일을 로드 


In [ ]:
def getPostData(post , jsonResult , cnt):
  title = post['title']#기사 제목
  description = post['description'] #네이버 기사 내용
  org_link = post['originallink'] # 원본 기사 내용 
  link = post['link']  #기사내용을 담고 있는 링크 
  #print(post)

  pDate = datetime.datetime.strptime(post['pubDate'],'%a, %d %b %Y %H:%M:%S +0900') #날짜와 시간 형식의 문자열을 datetime으로 변환하려면 strptime
  pDate = pDate.strftime('%Y-%m-%d %H:%M:%S') #날짜와 시간(datetime)을 문자열로 출력하려면 strftime
  #기사 내용의 정보를 dictionary 형태로 jsonResult 에 append
  jsonResult.append({'cnt':cnt,'title':title , 'description':description, 'org_link':org_link , 'link':link , 'pDate':pDate})
  return 

In [ ]:
def main():
  node = 'news'#네이버 뉴스 타이틀 검색 
  srcText = input('검색할 명령어를 입력하세요: ') #query 문 input
  cnt = 0 #기사 번호 index parameter
  jsonResult = [] #json file 결과 리스트 
  jsonResponse = getNaverSearch(node,srcText , 1 , 100) #node 와 query 문을 보내 검색 시작
  total = jsonResponse['total'] #반환 받은 json 파일의 
  while((jsonResponse !=None) and (jsonResponse['display'] !=0)): #검색된 기사의 내용이 끝날때까지  or url 를 받아올수 일을떄 까지 while 문을 사용하여 반복  
    for post in jsonResponse['items']:

      cnt +=1 #다음 기사로 넘어갈때 마다 연산을 통한 카운트 
      getPostData(post,jsonResult,cnt) #해당 기사의 정보를 getPostData 를 통해 jsonResult 의 저장 
    
    #기사의 시작 번호 
    start = jsonResponse['start'] + jsonResponse['display']
    
    #json file 의 정보 를 담기위해 검색한 쿼리 문 을 토대로 함수 호출
    jsonResponse = getNaverSearch(node,srcText,start,100)
    
  print('전체 검색 결과의수 : %d 건',total) # 검색되 기사의 수 

  #파일에 쓰기 위한 open 함수 
  '''
  open('파일명' , 'w/r' . encoding = 인코딩 할 언어) as '파일이름 '
  '''
  with open('%s_naver_%s.json' % (srcText,node),'w',encoding = 'utf-8') as outfile:
    #json.dumps = json 파일을 직렬화로 읽기 위한 변수 (indent = 들여쓰기 의 수 , sort_keys = 정렬 여부 , ensure_ascii = ascii 문자의 이스케이프 보장 여부 )
    jsonFile = json.dumps(jsonResult , indent = 4 , sort_keys = True , ensure_ascii=False)

    outfile.write(jsonFile) #jsonFile을 outfile 에 쓰기 
    print("가져온 데이터 : %d 건"%(cnt)) #가져온 기사의 데이터 수 
    print("%s_naver_%s.json SAVED" %(srcText,node)) #해당 검색을 통한 json 파일이 정상적으로 저장 됨을 알림

In [ ]:
if __name__ == '__main__': #main 함수 호출 실행 시킬려면 이름이 같아야함
  main()  #함수 호출 

검색할 명령어를 입력하세요: 월드컵


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



HTTP Error 400: Bad Request
[2022-04-06 06:35:30.306925] Error for URL: https://openapi.naver.com/v1/search/news.json?query=%EC%9B%94%EB%93%9C%EC%BB%B5&start=1001&display=100
전체 검색 결과의수 : %d 건 2406679
가져온 데이터 : 1000 건
월드컵_naver_news.json SAVED
